# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

C:\Users\ab9bh\Anaconda3\envs\AB_GPU\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [17]:
source_path = "E:/BigFiles/Project_data"

In [ ]:
def list_input_files(path = input_file_path,ext = '.pdf'):
    '''
    This function will lists the files based on the the Input file path & extension.
    input_file_path : Input Directory Path
    ext : extensiion of the file
        
    '''
    files_list = list()  
    for file in os.listdir(path):      
        if file.endswith(ext):
            files_list.append(file)

    return files_list

In [9]:
train_doc = np.random.permutation(open(source_path + "train.csv").readlines())
val_doc = np.random.permutation(open(source_path + "val.csv").readlines())
batch_size = 16


## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [4]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = ['WIN_20180907_15_35_09_Pro_Right Swipe_new','WIN_20180907_15_38_17_Pro_Left Swipe_new_Left Swipe_new']#create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = 4
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] = image - np.min(image)/np.max(image) - np.min(image)
                    batch_data[folder,idx,:,:,1] = image - np.min(image)/np.max(image) - np.min(image)
                    batch_data[folder,idx,:,:,2] = image - np.min(image)/np.max(image) - np.min(image)
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [26]:
folder_list=train_doc
t = np.random.permutation(folder_list)
t[0]

'WIN_20180907_16_35_51_Pro_Right Swipe_new;Right Swipe_new;1\n'

In [34]:
source_path+'/'+ t[0 + (0*batch_size)].strip().split(';')[0]+'/'

'E:/BigFiles/Project_data/train/WIN_20180925_17_50_59_Pro_Right_Swipe_new/'

In [30]:
image = imread(source_path+'/'+ t[0 + (0*batch_size)].strip().split(';')[0]+'/'+imgs['WIN_20180907_15_35_09_Pro_Right Swipe_new']).astype(np.float32)

TypeError: list indices must be integers or slices, not str

In [56]:
imread(source_path+'/'+ t[0 + (0*16)].strip().split(';')[0]+'/'+imgs[0]).astype(np.float32)

C:\Users\ab9bh\Anaconda3\envs\AB_GPU\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  """Entry point for launching an IPython kernel.


array([[[151., 153., 141.],
        [149., 151., 139.],
        [153., 155., 140.],
        ...,
        [154., 150., 133.],
        [152., 151., 133.],
        [151., 149., 132.]],

       [[153., 154., 142.],
        [150., 152., 140.],
        [151., 154., 139.],
        ...,
        [146., 142., 125.],
        [145., 144., 126.],
        [146., 145., 127.]],

       [[153., 155., 140.],
        [151., 154., 139.],
        [151., 154., 139.],
        ...,
        [146., 144., 129.],
        [148., 144., 127.],
        [147., 143., 126.]],

       ...,

       [[ 30.,  28.,  24.],
        [ 41.,  40.,  36.],
        [ 36.,  32.,  29.],
        ...,
        [ 31.,  29.,  23.],
        [ 32.,  31.,  24.],
        [ 32.,  31.,  24.]],

       [[ 32.,  28.,  25.],
        [ 44.,  40.,  37.],
        [ 36.,  32.,  29.],
        ...,
        [ 31.,  29.,  23.],
        [ 33.,  30.,  24.],
        [ 35.,  31.,  25.]],

       [[ 32.,  28.,  25.],
        [ 45.,  41.,  38.],
        [ 35.,  

In [54]:
imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0])
imgs[0]

'WIN_20180926_17_00_47_Pro_00000.png'

In [58]:
source_path = "E:/BigFiles/Project_data/train"
x = 3
y = 30
z = 30
folder_list=train_doc
t = np.random.permutation(folder_list)
#print('t',t)
img_idx = ['WIN_20180907_15_35_09_Pro_Right Swipe_new','WIN_20180907_15_38_17_Pro_Left Swipe_new_Left Swipe_new']
num_batches = 4
for batch in range(num_batches):
    batch_data = np.zeros((batch_size,x,y,z,3))
    batch_labels = np.zeros((batch_size,5))
    for folder in range(batch_size):
        imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0])
        for idx,item in enumerate(img_idx):
            image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[idx]).astype(np.float32)

C:\Users\ab9bh\Anaconda3\envs\AB_GPU\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()


In [36]:
curr_dt_time = datetime.datetime.now()
train_path = '/notebooks/storage/Final_data/Collated_training/train'
val_path = '/notebooks/storage/Final_data/Collated_training/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [37]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [7]:
optimiser = #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

SyntaxError: invalid syntax (<ipython-input-7-a1c9d2db73c4>, line 1)

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)